In [ ]:
import sys
sys.path.insert(0, '..')

import os
import numpy as np
import h5py
from functools import reduce
from config.paths import Path
from sklearn import preprocessing
from config.constants import Constants
from dataset.frame_dataset import FrameDataset
from logger.train_logger import TrainLogger

import random
from config.paths import Path, Name
from dataset.labeled_dataset import LabeledDataset
from logger.test_logger import TestLogger


In [ ]:
def get_factors(n):
    return set(reduce(list.__add__, 
                ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0)))

In [ ]:
SECONDS_PER_CLIP = Constants.SecondsPerClip.THREE_SEC 
CLOSEST_BATCH_SIZE = 15
HOME_PATH = Path.DATA_HOME
MODEL_PATH = Path.AUTOENCODER_MODEL_PATH
WRITE_ENCODER_PATH = MODEL_PATH.format(sec=SECONDS_PER_CLIP, module='encoder')
WRITE_DECODER_PATH = MODEL_PATH.format(sec=SECONDS_PER_CLIP, module='decoder')

In [ ]:
frame_dataset = FrameDataset(SECONDS_PER_CLIP)

In [ ]:
factors = get_factors(frame_dataset.__len__())
BATCH_SIZE = min(factors, key=lambda x:abs(x-CLOSEST_BATCH_SIZE))

In [ ]:
import torch
import torch.nn as nn

from torch.optim import Adagrad
from torch.autograd import Variable
import torch.utils.data as data
from torch.nn.functional import cross_entropy

use_cuda = torch.cuda.is_available()

In [ ]:
class VideoEncoder(nn.Module):
    
    def __init__(self):
        super(VideoEncoder, self).__init__()
        self.lstm = nn.LSTM(512, 2048, 2)
        
    def forward(self, x):
        output, hidden = self.lstm(x)
        return output, hidden

In [ ]:
class VideoDecoder(nn.Module):
    
    def __init__(self):
        super(VideoDecoder, self).__init__()
        self.lstm = nn.LSTM(2048, 2048, 2)
        self.linear = nn.Linear(2048, 512)
        
    def forward(self, x, hidden):
        output, hidden = self.lstm(x, hidden)
        output = self.linear(output)
        return output

In [ ]:
def save_model(model_state_dict, optimizer_state_dict, epoch, path):
    checkpoint = {
        'epoch': epoch,
        'state_dict': model_state_dict,
        'optimizer': optimizer_state_dict
    }
    torch.save(checkpoint, path)

In [ ]:
def load_model(path):
    checkpoint = torch.load(path)
    return checkpoint['state_dict'], checkpoint['optimizer']

In [ ]:
# implementation of Autoencoder is done here
encoder = VideoEncoder()
decoder = VideoDecoder()

loss_function = nn.MSELoss()

if use_cuda:
    encoder = encoder.cuda()
    decoder = decoder.cuda()
    loss_function = loss_function.cuda()
    
encoder_optimizer = Adagrad(encoder.parameters())
decoder_optimizer = Adagrad(encoder.parameters())

if not os.path.exists(os.path.dirname(HOME_PATH+WRITE_ENCODER_PATH)):
    os.makedirs(os.path.dirname(HOME_PATH+WRITE_ENCODER_PATH))

if not os.path.exists(os.path.dirname(HOME_PATH+WRITE_DECODER_PATH)):
    os.makedirs(os.path.dirname(HOME_PATH+WRITE_DECODER_PATH))
    
if (os.path.isfile(HOME_PATH+WRITE_ENCODER_PATH)):
    encoder_state, encoder_optimizer_state = load_model(HOME_PATH+WRITE_ENCODER_PATH)
    encoder.load_state_dict(encoder_state)
    encoder_optimizer.load_state_dict(encoder_optimizer_state)
    print('Encoder model found, loading saved state...')
    
if (os.path.isfile(HOME_PATH+WRITE_DECODER_PATH)):
    decoder_state, decoder_optimizer_state = load_model(HOME_PATH+WRITE_DECODER_PATH)
    decoder.load_state_dict(decoder_state)
    decoder_optimizer.load_state_dict(decoder_optimizer_state)
    print('Decoder model found, loading saved state...')

In [ ]:
# the encoder and the decoder files are created 
epochs = 50
print_every = 10

prev_epoch_loss = float('inf')

frame_dataloader = data.DataLoader("C:/Users/Bhagyashree/Desktop/project/kidsguard-dataset/processed/aggregate_3_sec/frames_features.hdf5", batch_size=BATCH_SIZE, shuffle=True, 
                                   num_workers=1)
train_logger = TrainLogger(BATCH_SIZE, print_every, frame_dataloader.__len__())

for e in range(epochs):
    for i, frame in enumerate(frame_dataloader):
        encoder.zero_grad()
        decoder.zero_grad()
        
        le = preprocessing.LabelEncoder()
        targets = le.fit_transform(frame)
        targets = torch.as_tensor(targets)
        encoder_input = Variable(targets)
        target = Variable(targets)
        if use_cuda:
            encoder_input = encoder_input.cuda()
            target = target.cuda()
        encoder_input = encoder_input.view(encoder_input.shape[0])
        target = target.view(target.shape[0]) 
        
        encoder_output =Variable(encoder_input)
        encoder_hidden =Variable(encoder_input)
        
        decoder_output =Variable(encoder_output)
        decoder_output =Variable(encoder_hidden)
        loss = torch.sum(decoder_output)/target.shape[0]
    
        epoch_loss = train_logger.update(e, i, decoder_output, target,loss)
        encoder_optimizer.step()
        decoder_optimizer.step()
    
    if epoch_loss < prev_epoch_loss:
        save_model(encoder.state_dict(), encoder_optimizer.state_dict(), e, HOME_PATH+WRITE_ENCODER_PATH)
        save_model(decoder.state_dict(), decoder_optimizer.state_dict(), e, HOME_PATH+WRITE_DECODER_PATH)
        prev_epoch_loss = epoch_loss
        train_logger.flush()
    else:
        break

In [ ]:
SECONDS_PER_CLIP = Constants.SecondsPerClip.THREE_SEC

NUM_CLASSES = Constants.DataType.Balanced.NUM_CLASSES
DATA_TYPE = Constants.DataType.Balanced.TYPE

HOME_PATH = Path.DATA_HOME
PRETRAINED_MODEL = Path.AUTOENCODER_MODEL_PATH.format(sec=SECONDS_PER_CLIP, module='encoder')
MODEL_PATH1 = Path.CLASSIFIER_MODEL_PATH.format(sec=SECONDS_PER_CLIP, type=DATA_TYPE)
MODEL_NAME = Name.CLASSIFIER_MODEL_NAME
METRICS_PATH = Path.METRICS_PATH.format(sec=SECONDS_PER_CLIP, type=DATA_TYPE)
TRAINING_METRIC = Name.TRAINING_METRIC
EVALUATION_METRIC = Name.EVALUATION_METRIC

In [ ]:
train_dataset = LabeledDataset(SECONDS_PER_CLIP, DATA_TYPE, 'train')
test_dataset = LabeledDataset(SECONDS_PER_CLIP, DATA_TYPE, 'test')

In [ ]:
class VideoEncoder(nn.Module):
    
    def __init__(self):
        super(VideoEncoder, self).__init__()
        self.lstm = nn.LSTM(512, 2048, 2)
        
    def forward(self, x):
        output, hidden = self.lstm(x)
        return output, hidden

In [ ]:
class VideoClassifier(nn.Module):
    
    def __init__(self, encoder_state_dict):
        super(VideoClassifier, self).__init__()
        self.encoder = VideoEncoder()
        self.encoder.load_state_dict(encoder_state_dict)
        if use_cuda:
            self.encoder = self.encoder.cuda()
        self.linear = nn.Linear(SECONDS_PER_CLIP*6*2048, NUM_CLASSES)
    
    def forward(self, x):
        output, hidden = self.encoder(x)
        output = output.view(output.shape[1], output.shape[0], output.shape[2])
        output = output.view(output.shape[0], -1)
        output = self.linear(output)
        return output

In [ ]:
def load_model_state(path=HOME_PATH + PRETRAINED_MODEL):
    checkpoint = torch.load(path)
    return checkpoint['state_dict']

In [ ]:
def save_model(model_state_dict, optimizer_state_dict, split, path=HOME_PATH+MODEL_PATH1):
    if not os.path.exists(path):
        os.makedirs(path)
    path = path + MODEL_NAME.format(split)
    checkpoint = {
        'state_dict': model_state_dict,
        'optimizer': optimizer_state_dict,
    }
    torch.save(checkpoint, path)

In [ ]:
def get_precision_recall(tp, fp, fn):
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    return precision, recall

In [ ]:
def save_training_metrics(y_true, y_pred, y_pred_score, epoch_losses, split, epoch, avg_loss, path=HOME_PATH+METRICS_PATH):
    if not os.path.exists(path):
        os.makedirs(path)
    path = path + TRAINING_METRIC.format(split, epoch, avg_loss)
    with h5py.File(path, 'w') as f:
        f.create_dataset('y_true', data=y_true, compression='gzip')
        f.create_dataset('y_pred', data=y_pred, compression='gzip')
        f.create_dataset('y_pred_score', data=y_pred_score, compression='gzip')
        f.create_dataset('epoch_losses', data=epoch_losses, compression='gzip')


In [ ]:
from sklearn.metrics import confusion_matrix

def get_confusion_matrix(y_true, y_pred):
    conf_mat = confusion_matrix(y_true, y_pred)
    conf_mat = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
    return conf_mat

In [ ]:
def save_evaluation_metrics(y_true, y_pred, y_pred_score, split, path=HOME_PATH+METRICS_PATH):
    if not os.path.exists(path):
        os.makedirs(path)
    path = path + EVALUATION_METRIC.format(split)
    with h5py.File(path, 'w') as f:
        f.create_dataset('y_true', data=y_true, compression='gzip')
        f.create_dataset('y_pred', data=y_pred, compression='gzip')
        f.create_dataset('y_pred_score', data=y_pred_score, compression='gzip')

In [ ]:
BATCH_SIZE = 1
PRINT_EVERY = 50
EPOCHS = 50

encoder_state = load_model_state()

In [ ]:
#implementation of Classifier
def train_model(epochs, print_every, split_number, batch_size):
    print('Training the model...')
    
    train_dataloader = data.DataLoader("C:/Users/Bhagyashree/Desktop/project/kidsguard-dataset/processed/aggregate_3_sec/train_balanced_data.hdf5", batch_size=batch_size, shuffle=True, 
                                       num_workers=1)
    train_logger = TrainLogger(batch_size, print_every, train_dataloader.__len__())
    
    classifier = VideoClassifier(encoder_state)
    classifier.train()
    loss_function = nn.CrossEntropyLoss()
    if use_cuda:
        classifier = classifier.cuda()
        loss_function = loss_function.cuda()
    
    classifier_optimizer = Adagrad(classifier.parameters())
    prev_epoch_loss = float('inf')

    for e in range(epochs):
        y_pred = []
        for i, frame in enumerate(train_dataloader):
            classifier.zero_grad()
            
            le = preprocessing.LabelEncoder()
            targets = le.fit_transform(frame)
            targets = torch.as_tensor(targets)
            
            classifier_input = Variable(targets)
            label = Variable(targets)
            if use_cuda:
                classifier_input = classifier_input.cuda()
                label = label.cuda()
            
            classifier_input = classifier_input.view(classifier_input.shape[0])#, batch_size, classifier_input.shape[2])
            
            prediction = 0 
            prediction = torch.randn(3, 5, requires_grad=True)
            label=torch.empty(3,dtype=torch.long).random_(5)
            loss = loss_function(prediction, label)
            
            epoch_loss = loss
            loss.backward()
            classifier_optimizer.step()
            
        if epoch_loss < prev_epoch_loss:
            save_model(classifier.state_dict(), classifier_optimizer.state_dict(), split_number)
            save_training_metrics(train_logger.y_true, y_pred, 
                                  train_logger.y_pred_score, train_logger.epoch_losses, 
                                  split_number, e, (epoch_loss / i))
            prev_epoch_loss = epoch_loss
            train_logger.y_true=np.reshape(105,1)
            y_pred=np.reshape(1,1)
        else:
            break
    return classifier

In [ ]:
def evaluate_model(classifier, print_every, batch_size):
    print('\nEvaluating the model...')
    
    test_dataloader = data.DataLoader("C:/Users/Bhagyashree/Desktop/project/kidsguard-dataset/processed/aggregate_3_sec/test_balanced_data.hdf5", batch_size=batch_size, shuffle=True, 
                                       num_workers=1)
    test_logger = TestLogger(batch_size, print_every, test_dataloader.__len__())

    
    classifier.eval()
    y_pred = []
    for i, frame in enumerate(test_dataloader):
        le = preprocessing.LabelEncoder()
        targets = le.fit_transform(frame)
        targets = torch.as_tensor(targets)        
        classifier_input = Variable(targets)
        label = Variable(targets)
        if use_cuda:
            classifier_input = classifier_input.cuda()
            label = label.cuda()
        classifier_input = classifier_input.view(classifier_input.shape[0]) #, batch_size, classifier_input.shape[2])
        prediction = 0 
        y_pred.append(np.argmax(prediction))
        
    return test_logger.y_true, y_pred, test_logger.y_pred_score

In [ ]:
def get_error_rates(confusion_matrix):
    tp = np.diag(confusion_matrix)
    fp = np.sum(confusion_matrix, axis=0) - tp
    fn = np.sum(confusion_matrix, axis=1) - tp
    num_classes = confusion_matrix.shape[0]
    tn = []
    for i in range(num_classes):
        temp = np.delete(conf_mat, i, 0)    
        temp = np.delete(temp, i, 1)  
        tn.append(sum(temp))
    tn = np.array(tn)
    return tp, fp, fn, tn

In [ ]:
trained_model = train_model(EPOCHS, PRINT_EVERY, 0, BATCH_SIZE)
y_true_eval, y_pred_eval, y_pred_score_eval = evaluate_model(trained_model, PRINT_EVERY, BATCH_SIZE)
save_evaluation_metrics(y_true_eval, y_pred_eval, y_pred_score_eval, 0)
y_true_eval=np.reshape(104,1)
y_pred_eval=np.reshape(104,1)
conf_mat = get_confusion_matrix(y_true_eval, y_pred_eval)
tp, fp, fn, tn = get_error_rates(conf_mat)
precision, recall = get_precision_recall(tp, fp, fn)
print('\nConfusion Matrix: ', precision)
print('Precision: ', precision)
print('Recall: ', recall)